In [ ]:
#in this version:
#removed parsing of xml to get all calculated fields and parameters as this can already be done with the doc api info
#removed doc api loop that only went through some sheets, as this was missing out some calculations. Now all calcs and default fields should be extracted

In [ ]:
import pandas as pd
import bs4 as bs
import zipfile
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os, re, sys

import win32com.client
from pywintypes import com_error
from os.path import isfile, join

import xml.etree.ElementTree as ET
from lxml import etree as ET

import tableaudocumentapi
from tableaudocumentapi import Workbook

import numpy as np

## Input folder - Find if there is a twbx or twb file in the folder
- if there is a twbx, unzip it to create a twb, then work with this
- if there's only a twb, work with this

In [ ]:
input_path = "inputs"
output_path = "outputs"

mypath = "./{}".format(input_path)

In [ ]:
#only gets files and not directories within the inputs folder -https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
f = [f for f in os.listdir(mypath) if isfile(join(mypath, f))] 
f

In [ ]:
def removeSpecialCharFromStr(spstring):
  
    return ''.join(e for e in spstring if e.isalnum())

In [ ]:
for i in [f for f in os.listdir(mypath) if isfile(join(mypath, f))] : 
   
    if i[-5:] == '.twbx':
        sp_packagedWorkbook = i[:len(i)-5]
        print(sp_packagedWorkbook)
        packagedWorkbook = removeSpecialCharFromStr(sp_packagedWorkbook)+'.twbx'
        print(packagedWorkbook)
        
        old_file = join(input_path, sp_packagedWorkbook+'.twbx')
        new_file = join(input_path, packagedWorkbook)
        os.rename(old_file, new_file)
        
        with zipfile.ZipFile(input_path+"/"+packagedWorkbook, 'r') as zip_ref:
            zip_ref.extractall(input_path+"/")
    else:
        packagedWorkbook = ""
        
for i in [f for f in os.listdir(mypath) if isfile(join(mypath, f))] :
    
    if i[-4:] == '.twb':
        sp_unpackagedWorkbook = i[:len(i)-4]
        unpackedWorkbook = removeSpecialCharFromStr(sp_unpackagedWorkbook)+'.twb' 
        
        old_file = join(input_path, sp_unpackagedWorkbook+'.twb')
        new_file = join(input_path, unpackedWorkbook)
        os.rename(old_file, new_file)

print('\n')
print('packaged workbook: ' + packagedWorkbook)
print('unpackaged workbook: ' + unpackedWorkbook)

In [ ]:
tableauFile = input_path+"/"+unpackedWorkbook
tableauFile

In [ ]:
packagedTableauFile = input_path+"/"+packagedWorkbook
packagedTableauFile

In [ ]:
#substring to be used when naming the exported data, based on the packaged workbook as this is what we download from tableau server
#we can rename the packaged workbook before running the script, giving more control to the naming of the extracted data

tableau_name_substring = packagedWorkbook.replace(".twbx","")[:30]
tableau_name_substring

# Doc API

In [ ]:
packagedTableauFile

In [ ]:
#trying new way to see if more default fields are extracted
sourceTWBX = Workbook(packagedTableauFile)

collator = []
calcID = []
calcID2 = []
calcNames = []

c = 0

worksheets = sourceTWBX.worksheets

for worksheet in worksheets:
    
    for datasource in sourceTWBX.datasources:
       
        for count, field in enumerate(datasource.fields.values()):
            
            #if worksheet in field.worksheets: #removed this part so all fields are listed,as otherwise some fields were missed out
                
                dict_temp = {}
                dict_temp['counter'] = c
                dict_temp['worksheet'] = worksheet
                dict_temp['datasource_name'] = datasource.name
                dict_temp['field_WHOLE'] = field
                dict_temp['field_name'] = field.name
                dict_temp['field_caption'] = field.caption
                dict_temp['field_calculation'] = field.calculation
                dict_temp['field_id'] = field.id
                dict_temp['field_datatype'] = field.datatype
                
                
                if not(isinstance(field.calculation, type(None))):
                    calcID.append(field.id)
                    calcNames.append(field.name)
                    
                    f2 = (field.id).replace(']','')
                    f2 = f2.replace('[', '')
                    calcID2.append(f2)
                
                c = c + 1
                
                collator.append(dict_temp)

In [ ]:
calcDict = dict(zip(calcID, calcNames))
calcDict2 = dict(zip(calcID2, calcNames))

In [ ]:
def default_to_friendly_names(formulaList,fieldToConvert, dictToUse):

    for i in formulaList:
        for tableauName, friendlyName in dictToUse.items():
            try:
                i[fieldToConvert] = (i[fieldToConvert]).replace(tableauName, friendlyName)
            except:
                a = 0
       
    return formulaList

In [ ]:
default_to_friendly_names(collator,'field_calculation',calcDict)

df_fields_DocAPI = pd.DataFrame(collator)

def f(row):
    if row['field_calculation'] == None:
        val = 'Datasource field'
    else:
        val = 'Calculated field'
    return val

df_fields_DocAPI['field_type'] = df_fields_DocAPI.apply(f, axis=1)

df_fields_DocAPI.head()

In [ ]:
#df_fields_DocAPI[colsToUse][(df_fields_DocAPI['field_type']!='Datasource field')]

# Calculated fields and parameters

In [ ]:
colsToUse = ['field_id', 'field_name', 'field_calculation', 'field_caption','field_datatype', 'datasource_name' ]

In [ ]:
#get parameters from doc api
len_parameters = len(df_fields_DocAPI[colsToUse][(df_fields_DocAPI['field_type']!='Datasource field') & (df_fields_DocAPI['datasource_name']=='Parameters')].drop_duplicates())
print('Parameter count: ' + str(len_parameters))

In [ ]:
#get calculated fields from doc api
len_CalcFields = len(df_fields_DocAPI[colsToUse][(df_fields_DocAPI['field_type']!='Datasource field') & (df_fields_DocAPI['datasource_name']!='Parameters')].drop_duplicates())
print('Calculated field count: ' + str(len_CalcFields))

In [ ]:
dfAPIcalcsAndParameters = df_fields_DocAPI[colsToUse][df_fields_DocAPI['field_type']!='Datasource field'].drop_duplicates().copy()
dfAPIcalcsAndParameters['isParameter'] = np.where(dfAPIcalcsAndParameters['datasource_name']=='Parameters', 'yes', 'no')

df = dfAPIcalcsAndParameters[['field_name', 'field_datatype', 'field_calculation','isParameter', 'field_id']].copy()
df.columns = ['CalculationName', 'DataType', 'Formula', 'isParameter', 'RawName']

df = df.sort_values(by=['isParameter','CalculationName'])
df = df.reset_index(drop=True)

df

# All default fields

In [ ]:
#default fields
len_defaultFields = len(df_fields_DocAPI[colsToUse][df_fields_DocAPI['field_type']=='Datasource field'].drop_duplicates())
print('Default field count: ' + str(len_defaultFields))

In [ ]:
df_defaultFields = df_fields_DocAPI[['field_id', 'field_caption','field_datatype', 'datasource_name' ]][df_fields_DocAPI['field_type'] == 'Datasource field'].drop_duplicates()

df_defaultFields['prefOrder'] = np.where(df_defaultFields['field_caption'].isnull(), 0, 1)

df_defaultFields['field_id2'] = df_defaultFields['field_id'].str.replace('[','')
df_defaultFields['field_id2'] = df_defaultFields['field_id2'].str.replace(']','')

df_defaultFields = df_defaultFields.sort_values(by = ['field_id2'])

#https://stackoverflow.com/questions/63271050/use-drop-duplicates-in-pandas-df-but-choose-keep-column-based-on-a-preference-li
preference_list=[1,0]

df_defaultFields["prefOrder"] = pd.Categorical(df_defaultFields["prefOrder"], categories=preference_list, ordered=True)

df_defaultFields = df_defaultFields.sort_values(["field_id2","prefOrder"]).drop_duplicates("field_id2")
df_defaultFields = df_defaultFields.drop('prefOrder', axis=1)
df_defaultFields = df_defaultFields.drop('field_id2', axis=1)
df_defaultFields

# Sheet - all field dependencies, not just the explicitly used fields

In [ ]:
df_sheetDependencies = df_fields_DocAPI.copy()

df_sheetDependencies['prefOrder'] = np.where(df_sheetDependencies['field_caption'].isnull(), 0, 1)

df_sheetDependencies['field_id2'] = df_sheetDependencies['field_id'].str.replace('[','')
df_sheetDependencies['field_id2'] = df_sheetDependencies['field_id2'].str.replace(']','')

preference_list=[1,0]

df_sheetDependencies["prefOrder"] = pd.Categorical(df_sheetDependencies["prefOrder"], categories=preference_list, ordered=True)
df_sheetDependencies = df_sheetDependencies.sort_values(["field_id2","prefOrder"]).drop_duplicates(subset=["field_id2", "worksheet"])
df_sheetDependencies = df_sheetDependencies.drop('prefOrder', axis=1)
df_sheetDependencies = df_sheetDependencies.drop('field_id2', axis=1)

df_sheetDependencies = df_sheetDependencies.drop(columns=['counter', 'field_caption', 'field_WHOLE', 'field_calculation', 'field_id'])

df_sheetDependencies = df_sheetDependencies[['worksheet', 'field_name', 'field_datatype', 'field_type', 'datasource_name']].sort_values(by = ['worksheet', 'field_type','datasource_name', 'field_name'])
df_sheetDependencies

# Getting all filters for all worksheets

In [ ]:
# for worskheet in root.findall("./worksheets/worksheet"):
#     for filt in worskheet.findall('table/view/filter'):
#         print(filt)

In [ ]:
tree = ET.parse(tableauFile)
root = tree.getroot()

In [ ]:
filters_in_sheet = []
context = []
collatelist = []

for worskheet in root.findall("./worksheets/worksheet"):
    
    tempdict = {}
    c = 0
    
    for filt in worskheet.findall('table/view/filter'):

        calcfromfilter = filt.attrib['column']        
        pat = '(?<=\:)(.*?)(?=\:)' 
        string_cleaned = calcfromfilter.split('].[')[1].replace(']','')
        
        tempdict['field'] = calcfromfilter
        tempdict['formula'] = calcfromfilter
        tempdict['counter'] = c
        tempdict['sheetname'] = worskheet.attrib['name']
        
        try:
            st1 = re.findall(pat,string_cleaned)[0]
            tempdict['field'] = st1
            tempdict['formula'] = st1
            collatelist.append(tempdict)
            
        except:
            st2 = string_cleaned.replace(':','')
            tempdict['field'] = st2
            tempdict['formula'] = st2
            collatelist.append(tempdict)

        try:
            tempdict['context'] = filt.attrib['context']
        except:
            tempdict['context'] = 'False'
           
        c = c + 1
        tempdict = {}
    
collatelist[0:2]

In [ ]:
collatelist = default_to_friendly_names(collatelist, 'formula', calcDict2)
collatelist[0:2]

In [ ]:
try: 
    df1 = pd.DataFrame(collatelist)

    df1 = df1[['sheetname', 'formula', 'context', 'field']]
    df1.columns = ['Sheet Name', 'FilterField', 'Context filter', 'FilterField_RawName']

    print(df1.head(2))
except:
    print('error with df1')

# Extracting rows and cols for each sheet

In [ ]:
collecteddata = []

for worksheet in root.findall("./worksheets/worksheet"):

    argumentstopass = ['rows', 'cols']
    
    for i in argumentstopass:   
    
        internaldict = {}

        internaldict['sheetname'] = worksheet.attrib['name']
        internaldict['type'] = i
        
        formulahere = worksheet.findall('table/'+i)[0].text
        internaldict['formula'] = formulahere
        
        collecteddata.append(internaldict)
    
collecteddata[0:2]

In [ ]:
for i in collecteddata:

    try:
        pattern = '\:.*?\:'
        pat = '(?<=\:)(.*?)(?=\:)'

        calculationsWithColon = re.findall(pattern,i['formula']) 
        calcsWithoutColon = []

        for n in calculationsWithColon:
            oneCalcWithoutColon = re.findall(pat,n)[0]

            calcsWithoutColon.append(oneCalcWithoutColon)

        i['extracted formulas'] = calcsWithoutColon
        
    except:
        i['extracted formulas'] = []
             
    newcalcs = []
    formulas_to_process = i['extracted formulas']
    
    for n in formulas_to_process:

        for tableauName, friendlyName in calcDict2.items():
            
            n = n.replace(tableauName, friendlyName)
        newcalcs.append(n)
        
    i['processed formulas'] = newcalcs

collecteddata[0:2]

In [ ]:
df2 = pd.DataFrame(collecteddata)
df2 = df2.drop(columns=['formula', 'extracted formulas'])
df2 = df2.pivot(index='sheetname', columns='type', values='processed formulas')
df2 = df2.reset_index()
df2.head(2)

# General workbook description

In [ ]:
sourceTWBX = Workbook(packagedTableauFile)

In [ ]:
collate_list = []

for dash in sourceTWBX.dashboards:
    dicti = {}
    
    dicti['type'] = 'dashboard'
  #  print(format(dash))
    dicti['name'] = format(dash)
   
    collate_list.append(dicti)
    
for data in sourceTWBX.datasources:
    dicti = {}
    
    dicti['type'] = 'datasource'
    dicti['name'] = format(data.name)
   # print(format(data.name))
   
    collate_list.append(dicti)
    
for data in sourceTWBX.worksheets:
    dicti = {}
    
    dicti['type'] = 'sheet'
    dicti['name'] = format(data)
   # print(format(data))
    
    collate_list.append(dicti)

In [ ]:
df_workbookdec = pd.DataFrame(collate_list)
df_workbookdec = df_workbookdec[['type', 'name']]
df_workbookdec.head(2)

In [ ]:
df_workbookdec_counts = df_workbookdec.groupby(['type']).count().reset_index()
df_workbookdec_counts

## Generating an excel file from a df (so the excel rows/cols can be formatted), then turning the excel into a pdf

In [ ]:
#load env variable with full path to folder, as otherwise the pdf creation fails

with open("archive/path_string.txt", "r") as f:
    os.environ["path_string"] = f.read()
    
path_string = os.environ.get('path_string', 'Not Set')
path_string = r"{}".format(path_string)    #must create literal ending in tableauCalculationExport\{}

- Loading the file names and output locations for the excel and pdfs to be produced

In [ ]:
name_to_use = tableau_name_substring    

newFileName = 'outputs\{}'.format(name_to_use)
excelName = newFileName + ".xlsx"
pdfName = newFileName + ".pdf"
print(pdfName)

excel_path = path_string.format(excelName)
path_to_pdf = path_string.format(pdfName)

- Functions to format the excel files

In [ ]:
#colors to be used in each sheet
c1 = '#f4dfa4'
c2 = '#ffc8b3'
c3 = '#fff0b3'
c4 = '#d5dfb9'
c5 = '#d1c5d3'
c6 = '#bfd9d7'

In [ ]:
def mainCol(colNumber, color):
    format_mainCol = workbook.add_format({'text_wrap': True, 'bold': True})
    format_mainCol.set_align('vcenter')
    format_mainCol.set_bg_color(color)
    format_mainCol.set_border(1)
    worksheet.set_column(colNumber,colNumber,20,format_mainCol)
    return worksheet

In [ ]:
def normalCol(colNumber, colWidth):
    format2 = workbook.add_format({'text_wrap': True})
    format2.set_align('vcenter')
    format2.set_border(1)
    worksheet.set_column(colNumber,colNumber,colWidth,format2)
    return worksheet

- Creation of excel file

In [ ]:
#modify this part if you want to add more information/dfs to be saved as a separate sheet in excel

dfs_to_use = [{'excelSheetTitle': 'Dashboard, datasource and sheet details', 'df_to_use':df_workbookdec, 'mainColWidth':'' , 
               'normalColWidth': [30], 'sheetName': 'GeneralDetails', 'footer': 'Data_1', 'papersize':9, 'color': c1} , 
              
              {'excelSheetTitle': 'Overall counts of dashboards, datasources and sheets', 'df_to_use':df_workbookdec_counts, 'mainColWidth':'' , 
               'normalColWidth': [10], 'sheetName': 'GeneralCounts', 'footer': 'Data_2', 'papersize':9, 'color': c1},
              
              {'excelSheetTitle': 'Default fields from all datasources', 'df_to_use':df_defaultFields, 'mainColWidth':'' , 
               'normalColWidth': [20,20,40], 'sheetName': 'DefaultFields', 'footer': 'Data_3', 'papersize':9, 'color': c2},
              
              {'excelSheetTitle': 'Calculated fields and parameters', 'df_to_use':df, 'mainColWidth':'' , 
               'normalColWidth': [10,50,10,20], 'sheetName': 'CalculatedFields', 'footer': 'Data_4', 'papersize':9, 'color': c3},
              
              {'excelSheetTitle': 'Filters used in each sheet', 'df_to_use':df1, 'mainColWidth':'' , 
               'normalColWidth': [20,20,40], 'sheetName': 'Filters', 'footer': 'Data_5', 'papersize':9, 'color': c4},
              
              {'excelSheetTitle': 'Metrics used in Columns and Rows, for each sheet', 'df_to_use':df2, 'mainColWidth':'' , 
               'normalColWidth': [30,40], 'sheetName': 'RowsAndCols', 'footer': 'Data_6', 'papersize':9, 'color': c5},
              
              {'excelSheetTitle': 'Sheet dependencies on default fields, calculated fields and parameters', 'df_to_use':df_sheetDependencies, 'mainColWidth':'' , 
               'normalColWidth': [30,15,25,30], 'sheetName': 'SheetDependencies', 'footer': 'Data_7', 'papersize':8, 'color': c6}
             ]

#papersize: a3 = 8, a4 = 9

In [ ]:
writer = pd.ExcelWriter(excelName, engine = 'xlsxwriter')

#code to create each sheet in excel, with the specified df and formatting each sheet as per requirements
#also adds a header and footer to each sheet
#all the info to be replaced below (ie. for each df) comes form the dfs_to_use list of dictionaries

for x in dfs_to_use:
    excelSheetTitle = x['excelSheetTitle']
    df_to_use = x['df_to_use']
    normalColWidth = x['normalColWidth']
    sheetName = x['sheetName']
    papersize = x['papersize']
    footer = x['footer']
    color = x['color']

    df_to_use.to_excel(writer, sheet_name = sheetName, index=False)
    
    workbook=writer.book
    worksheet = writer.sheets[sheetName]

    worksheet = mainCol(0, color)
    
    ws = 1
    for i in normalColWidth:
        worksheet = normalCol(ws,i)
        ws = ws + 1

    worksheet.set_paper(papersize) # a4
    worksheet.fit_to_pages(1,0)    # fit to 1 page wide, n long
    worksheet.repeat_rows(0)       # repeat the first row
    
    header_x = '&C&"Arial,Bold"&10{}'.format(excelSheetTitle)
    footer_x = '&L{}&CPage &P of &N'.format(footer)

    worksheet.set_header(header_x)
    worksheet.set_footer(footer_x)

writer.save()

- Creation of pdf from excel file

In [ ]:
#this creates an index to list each excel sheet, based on the number of sheets that were created before

for_ws_index_list = []
for i in range(len(dfs_to_use)):
    for_ws_index_list.append(i+1)

In [ ]:
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

wb = excel.Workbooks.Open(excel_path)

#print all the excel sheets into a single pdf
ws_index_list = for_ws_index_list
wb.Worksheets(ws_index_list).Select()
wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
wb.Close()
excel.Quit()